# Simplex for transportation problem

We are going to create a simplex algorithm adept for transportation problems and we are going to use the matrix minization method to compute initial solutions for the initial problem. In the transportation problem we have a network of $n$ supply vertex and $m$ demand vertex, our variables $x_{i,j}$ indicate how much is being transported from vertex $i$ to vertex $j$. We have two vectors, $s$ and $d$ with the maximum values for supply and demand for each vertex. The general formulation is:

$$\min_{x} x^Tc$$

$$\text{subject to } $$

$$\sum_{j = 1}^{m} x_{i, j} = s_i$$

$$\sum_{i = 1}^{n} x_{i, j} = d_i$$

The adaptions made in order to focus on transport problems and get better processing time are:

- The upper bound constraints are removed, we only have equality constraints.

- Without upper bound constraints, we don't have to deal with slack variables.

- We don't consider negative right side (there is no negative supply or demand).

- We need an artificial variable in every constraint because there is no isolated variable (every variable x_ij is in the constraint for the supply vertex i and also in the constraint for the demand vertex j).

- The problem is for minimization.



In [1]:
using LinearAlgebra

mutable struct Tableau
    T::Array{Float64, 2}
    basis::Dict{Int64,Int64}
    decision::Array{Int64, 1}
    slack::Array{Int64, 1}
    artificial::Array{Int64, 1}
    constraints::Array{Int64, 1}
    iterations::Int64
end

function add_artificial(T::Array{Float64, 2},
                    n_const::Int64,
                    basis::Dict{Int64, Int64})
    """
    Function that add artificial variables to every row
    """
    artificial = []
    aux = zeros(1, size(T, 2))
    T = [T; aux]
    for i = 1:(n_const)
        aux = zeros(size(T, 1))
        aux[i] = 1.0
        T = [T[:, 1:(end- 1)] aux T[:, end]]
        artificial = [artificial; (size(T, 2) - 1)]
        push!(basis, i => last(artificial))
        T[end, :] = T[end, :] + T[i, :]
    end
    
    T[end, (end - size(artificial, 1)):(end - 1)] .= 0
    return T, basis, artificial
end

function pivot_tableau(tableau::Tableau,
                        r::Int64,
                        s::Int64)
    """
    Function that pivot tableau, removing variable of line r from basis and adding variable s to line r
    """
    
    #update basis variable
    delete!(tableau.basis, r)
    push!(tableau.basis, r => s)   
    
    #normalize row r
    tableau.T[r, :] .= tableau.T[r, :]/tableau.T[r, s]
    
    #subtract from others rows
    for i=1:size(tableau.T, 1)
        if i != r
            tableau.T[i, :] .= tableau.T[i, :] .- (tableau.T[i, s] * tableau.T[r, :])
        end
    end
    
    return tableau
end

function simplex_iteration(tableau::Tableau, max_int::Int64)
    """
        Function that run the iterations of optimization for the simplex
    
        tableau - tableau object with problem information
        max_int - number of max iterations
        
    """
    n = size(tableau.T, 1)
    m = size(tableau.T, 2)
    
    for int=1:max_int
        #can't increase objective
        if maximum(tableau.T[n, 1:(end - 1)]) <= 0
            return tableau
        end
        
        #variable to enter basis
        s = argmax(tableau.T[n, 1:(end -1)])
        
        #check if is unbounded
        if maximum(tableau.T[tableau.constraints,s]) <= 0
            error("Problem is unbounded.")
        end
        
        
        #ratio test to find variable to remove from basis
        r = -1
        min_ratio = Inf
        for i in tableau.constraints
            if tableau.T[i, s] > 0
                new_ratio = tableau.T[i, m]/tableau.T[i, s]
                if new_ratio < min_ratio
                    r = i
                    min_ratio = new_ratio
                end
            end
        end
        tableau = pivot_tableau(tableau, r, s)
        tableau.iterations = int
    end
    
    return tableau
end


mutable struct solution
    objective::Float64
    decision::Array{Float64, 1}
    iterations::Int64
end

function create_solution(tableau::Tableau)
    objective = tableau.T[end, end]
    x = zeros(size(tableau.decision, 1))
    for (key, value) in tableau.basis
        if value in tableau.decision
            x[value] = tableau.T[key, end]
        end
    end
    return solution(objective, x, tableau.iterations)
end

function linear_program(c::Array{Float64,1},
                        A::Array{Float64,2},
                        b::Array{Float64,1},
                        max_int::Int64,
                        verbose::Bool,
                        X::Array{Float64, 1})
    
    
    n_variables = size(A, 2)
    n_const = size(A, 1)
    decision = collect(1:n_variables)
    constraint = collect(1:n_const)
    c = -1 * c
    if verbose
        println(string(n_variables) * " decision variables.")
        println(string(n_const) * " constraints.")
    end

    T = zeros((n_const + 1), (n_variables + 1))
    T[1:(end -1), :] = [A b]
    T[end, :] = [c; 0]
    
    basis = Dict{Int64,Int64}()
    
    T, basis, artificial = add_artificial(T, n_const, basis)
    
    if verbose
        println(string(size(artificial, 1)) * " artificial variables.")
        println("Basis:")
        println(basis)
    end
    
        
    tableau = Tableau(T, basis, decision, [], artificial, constraint, 0) 
    
    
    inds = []
    for i = 1:size(X, 1)
        if X[i] > 0
            inds = [inds; i]
        end
    end
    
    if verbose
        println("Variables with non-zero values: " * string(inds))
    end
    
    
    k = 1
    while k <= size(inds, 1)
        var_to_enter = inds[k]
        possible_rows = []
        for i = 1:n_const
            if tableau.T[i, var_to_enter] == 1
                possible_rows = [possible_rows; i]
            end
        end
        
        for i in possible_rows
            if !(tableau.basis[i] in inds[1:k])
                pivot_tableau(tableau, i, var_to_enter)
                break
            end
        end
        k = k + 1
    end
    
    tableau.T = tableau.T[1:(end - 1), setdiff(1:size(tableau.T, 2), tableau.artificial)]
    
    if verbose
        println("Basis after updated basis with initial values:")
        println(tableau.basis)
    end
    
    tableau = simplex_iteration(tableau, max_int)
    
    return create_solution(tableau)
end

linear_program (generic function with 1 method)

We create the matrix minimization method, in this method we focus on finding the limit for each constraint based on the current lowest cost. 

In [2]:
function matrix_method(cost, demand, supply)
    #In the cost matrix, each row represents a supply, each column represents a demand
    #The quantitys for each vertex to be completed
    demand_remaining = [d for d in demand]
    supply_remaining = [s for s in supply]
    #Indicating if some vertex if already fullfield
    demand_bool = [false for _ in demand]
    supply_bool = [false for _ in supply]
    #Initial variable
    x0 = [0 for _ in cost]
    value, edge = findmin(cost)
    
    while sum(demand_remaining) + sum(supply_remaining) > 0
        cur_supply = getindex(edge, 1)
        cur_demand = getindex(edge, 2)
        #Can only move the minimun between the supply and demand 
        edge_value = minimum([demand_remaining[cur_demand], supply_remaining[cur_supply]])
        
        #Update vectors
        demand_remaining[cur_demand] -= edge_value
        supply_remaining[cur_supply] -= edge_value
        x0[cur_supply, cur_demand] = edge_value
        
        #Check if the updated vertex is complete
        if demand_remaining[cur_demand]  == 0
            demand_bool[cur_demand] = true
        end
        if supply_remaining[cur_supply] == 0
            supply_bool[cur_supply] = true
        end
        
        #Little trick to don't find arcs of completed vertex
        cost[supply_bool, :] .= Inf
        cost[:, demand_bool] .= Inf
        value, edge = findmin(cost)
    end
    return x0
end

matrix_method (generic function with 1 method)

We also have to adapt our problem, currently $x$ is a $2D$ variable, we have to represent this matrix of variables in a 1D vector. For this, we are going to place variable $i, j$ at index $(j + (i - 1) \cdot n)$ with $n$ being the number of rows. The costs matrix must also be flattened with the same index as the vraiables matrix.

We also want to build the restrictions for the demand and supply arrays, for each vertex we need to create a restriction. We have a total of $n + m$ vertex, $n$ for supply and $m$ for demand, so $A$ will have size $[n + m, n\cdot m]$ and $b$ will have size $[n + m]$.

In [3]:
function flatten_problem(x0::Array{Float64, 2}, 
                        costs::Array{Float64, 2},
                        supply::Array{Float64, 1}, 
                        demand::Array{Float64, 1}) 
    n, m = size(x0)
    x0_flat = zeros(n * m)
    costs_flat = zeros(n * m)
    for i = 1:n
        for j = 1:m
            x0_flat[j + (i - 1) * n] = x0[i, j]
            costs_flat[j + (i - 1) * n] = costs[i, j]
        end
    end
    
    b = [supply; demand]
    A = zeros(n+m, n*m)
    
    #supply restrictions
    for i = 1:n
        row = zeros(n * m)
        for j = 1:m
            row[j + (i - 1) * n] = 1
        end
        A[i, :] .= row
    end
    
    #demand restriction
    for j = 1:m
        row = zeros(n * m)
        for i = 1:n
            row[j + (i - 1) * n] = 1
        end
        A[(n+j), :] .= row
    end
    return x0_flat, costs_flat, A, b
end

flatten_problem (generic function with 1 method)

## Example

For a simple example of the method we are going to use exercise 3 from chapter 8 of AMP. In this problem we have 5 demand vertex and 5 supply vertex. With the demand vector, supply vector and costs matrix we are going to initially compute a initial solution using matrix minimization method.

In [4]:
costs = [6. 6 9 4 10;
3 2 7 5 12;
8 7 5 6 4;
11 12 9 5 2;
4 3 4 5 11]
demand = [120, 80, 50, 75, 85]
supply = [45, 90, 95, 75, 105]

x0 = matrix_method(copy(costs), copy(demand), copy(supply))
x0_flat, c, A, b = flatten_problem(float(x0), costs, float(supply), float(demand))

x0

5×5 Array{Int64,2}:
   0   0   0  45   0
  10  80   0   0   0
   5   0  50  30  10
   0   0   0   0  75
 105   0   0   0   0

In [5]:
sol = linear_program(c, A, b, 100, true, x0_flat)
println("Z = " * string(sol.objective))
println("X = " * string(sol.decision))
println("Number of iterations: " * string(sol.iterations))

25 decision variables.
10 constraints.
10 artificial variables.
Basis:
Dict(7 => 32,4 => 29,9 => 34,10 => 35,2 => 27,3 => 28,5 => 30,8 => 33,6 => 31,1 => 26)
Variables with non-zero values: Any[4, 6, 7, 11, 13, 14, 15, 20, 21]
Basis after updated basis with initial values:
Dict(7 => 7,4 => 20,9 => 14,10 => 15,2 => 6,3 => 11,5 => 21,8 => 13,6 => 31,1 => 4)
Z = 1450.0
X = [0.0, 0.0, 0.0, 45.0, 0.0, 10.0, 80.0, 0.0, 0.0, 0.0, 5.0, 0.0, 50.0, 30.0, 10.0, 0.0, 0.0, 0.0, 0.0, 75.0, 105.0, 0.0, 0.0, 0.0, 0.0]
Number of iterations: 0


## Example with big problem

We create a function with given number of supply and demand vertex, create a fake supply, demand and costs arrays.

In [26]:
function generate_fake(n::Int64, m::Int64)
    X = zeros(n, m)
    c = zeros(n, m)
    
    for i = 1:n
        for j = 1:m
            not_zero = rand() > 0.6
            if not_zero
                X[i, j] = floor(rand() * 100)
            end
            
            big_cost = rand() > 0.8
            if big_cost
                c[i, j] = floor(rand() * 20) + 2
            else
                c[i, j] = floor(rand() * 5) + 1
            end
        end
    end
    
    supply = zeros(n)
    demand = zeros(m)
    
    for i = 1:n
        supply[i] = sum(X[i, :])
    end
    
    for j = 1:m
        demand[j] = sum(X[:, j])
    end
    
    return X, c, supply, demand
end

generate_fake (generic function with 1 method)

__With 100 variables:__

In [27]:
X, costs, supply, demand = generate_fake(10, 10)
x0 = matrix_method(copy(costs), copy(demand), copy(supply))
x0_flat, c, A, b = flatten_problem(float(x0), costs, float(supply), float(demand));

In [28]:
sol = linear_program(c, A, b, 100, false, x0_flat)
println("Number of iterations: " * string(sol.iterations))
println("Objective: " * string(sol.objective))

Number of iterations: 3
Objective: 3242.0


In [29]:
using JuMP
using Cbc

m = Model(Cbc.Optimizer)

@variable(m, x[1:10, 1:10] >= 0)
set_start_value.(x, x0)

@constraint(m, const_demand[i=1:10], sum(x[:, i]) == demand[i])

@constraint(m, const_supply[i=1:10], sum(x[i, :]) == supply[i])

@objective(m, Min, sum(x .* costs))
optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 20 (0) rows, 100 (0) columns and 200 (0) elements
Perturbing problem by 0.001% of 18 - largest nonzero change 8.8166274e-05 ( 0.0018434118%) - largest zero change 0
0  Obj 0 Primal inf 4324 (20)
20  Obj 3242.034
Optimal - objective value 3242
Optimal objective 3242 - 20 iterations time 0.002
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



__With 10000 variables__:

In [30]:
X, costs, supply, demand = generate_fake(100, 100)
x0 = matrix_method(copy(costs), copy(demand), copy(supply))
x0_flat, c, A, b = flatten_problem(float(x0), costs, float(supply), float(demand));

In [32]:
sol = linear_program(c, A, b, 100, false, x0_flat)
println("Number of iterations: " * string(sol.iterations))
println("Objective: " * string(sol.objective))

Number of iterations: 45
Objective: 200946.0


In [34]:
m = Model(Cbc.Optimizer)

@variable(m, x[1:100, 1:100] >= 0)
set_start_value.(x, x0)

@constraint(m, const_demand[i=1:100], sum(x[:, i]) == demand[i])

@constraint(m, const_supply[i=1:100], sum(x[i, :]) == supply[i])

@objective(m, Min, sum(x .* costs))
optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Presolve 200 (0) rows, 10000 (0) columns and 20000 (0) elements
Perturbing problem by 0.001% of 21 - largest nonzero change 9.999495e-05 ( 0.0020499219%) - largest zero change 0
0  Obj 0 Primal inf 401892 (200)
77  Obj 80992.914 Primal inf 317865.1 (161)
156  Obj 141459.57 Primal inf 322055 (128)
217  Obj 166549.94 Primal inf 236771.2 (114)
279  Obj 199798.23 Primal inf 116402 (76)
337  Obj 200948.27 Primal inf 96573.5 (77)
385  Obj 200948.28 Primal inf 4073.1 (32)
419  Obj 200948.28
Optimal - objective value 200946
Optimal objective 200946 - 419 iterations time 0.022
Total time (CPU seconds):       0.02   (Wallclock seconds):       0.02

